In [1]:
import os
def write_graph(name):
    logdir = os.path.join('log', name)
    file_writer = tf.summary.FileWriter(logdir)
    with tf.Session() as sess:
        file_writer.add_graph(sess.graph)

In [2]:
import sys
import numpy
import tensorflow as tf

import time

class DatasetReader(object):

    # TODO(student): You must implement this.
    @staticmethod
    def ReadFile(filename, term_index, tag_index):
        
        
        f = open(filename, "r")
        parsedFile = []
        indi = 0
        indj = 0
        
        if bool(term_index):
            m= max(term_index.values())
            indi = m+1


        for i in f:

            l = i.strip().split(" ")

            for j in l:

                w,t = j.rsplit("/",1)

                if w not in term_index:
                    if indi in term_index.values():
                        indi=indi+1
                    term_index[w] = indi
                    indi = indi + 1



                if t not in tag_index:
                    tag_index[t] = indj
                    indj = indj + 1



        f.close()
        ff = open(filename, "r")

        for it in ff:

            parsedLine = []

            ll = it.strip().split(" ")

            for jj in ll:
                word,tag = jj.rsplit("/",1)

                temp = (term_index[word], tag_index[tag])
                parsedLine.append(temp)

            parsedFile.append(parsedLine)
            
            
        
       
        """Reads file into dataset, while populating term_index and tag_index.
     
        Args:
            filename: Path of text file containing sentences and tags. Each line is a
                sentence and each term is followed by "/tag". Note: some terms might
                have a "/" e.g. my/word/tag -- the term is "my/word" and the last "/"
                separates the tag.
            term_index: dictionary to be populated with every unique term (i.e. before
                the last "/") to point to an integer. All integers must be utilized from
                0 to number of unique terms - 1, without any gaps nor repetitions.
            tag_index: same as term_index, but for tags.

        the _index dictionaries are guaranteed to have no gaps when the method is
        called i.e. all integers in [0, len(*_index)-1] will be used as values.
        You must preserve the no-gaps property!

        Return:
            The parsed file as a list of lists: [parsedLine1, parsedLine2, ...]
            each parsedLine is a list: [(termId1, tagId1), (termId2, tagId2), ...] 
            
            
        """
        return parsedFile


        pass
    

    # TODO(student): You must implement this.
    @staticmethod
    def BuildMatrices(dataset):
        
        
        
        l = [len(x) for x in dataset]
        length = max(l)

        terms_matrix = []
        tags_matrix = []
        for sentence in dataset:

            # lsmall=len(sentence)

            inlist = numpy.zeros((length),dtype=int)
            inlist_tag = numpy.zeros((length),dtype=int)

            for s in range(0, len(sentence)):
                inlist[s] = sentence[s][0]

            for ss in range(0, len(sentence)):
                inlist_tag[ss] = sentence[ss][1]

            terms_matrix.append(inlist)
            tags_matrix.append(inlist_tag)
            
        terms_matrix=numpy.array(terms_matrix,dtype=numpy.int64)
        tags_matrix=numpy.array(tags_matrix,dtype=numpy.int64)
        l=numpy.array(l,dtype=numpy.int64)


            
        print "....slength",l
        tuplee = (terms_matrix, tags_matrix, l)
        # terms_matrix= np.zeros((length))

        # tags_matrix=np.zeros((length))

        """Converts dataset [returned by ReadFile] into numpy arrays for tags, terms, and lengths.

        Args:
            dataset: Returned by method ReadFile. It is a list (length N) of lists:
                [sentence1, sentence2, ...], where every sentence is a list:
                [(word1, tag1), (word2, tag2), ...], where every word and tag are integers.

        Returns:
            Tuple of 3 numpy arrays: (terms_matrix, tags_matrix, lengths_arr)
                terms_matrix: shape (N, T) int64 numpy array. Row i contains the word
                    indices in dataset[i].
                tags_matrix: shape (N, T) int64 numpy array. Row i contains the tag
                    indices in dataset[i].
                lengths: shape (N) int64 numpy array. Entry i contains the length of
                    sentence in dataset[i].

            T is the maximum length. For example, calling as:
                BuildMatrices([[(1,2), (4,10)], [(13, 20), (3, 6), (7, 8), (3, 20)]])
            i.e. with two sentences, first with length 2 and second with length 4,
            should return the tuple:
            (
                [[1, 4, 0, 0],    # Note: 0 padding.
                 [13, 3, 7, 3]],

                [[2, 10, 0, 0],   # Note: 0 padding.
                 [20, 6, 8, 20]], 

                [2, 4]
            )
        """
        return tuplee
        pass

    @staticmethod
    def ReadData(train_filename, test_filename=None):
        """Returns numpy arrays and indices for train (and optionally test) data.

       

        Args:
            train_filename: .txt path containing training data, one line per sentence.
                The data must be tagged (i.e. "word1/tag1 word2/tag2 ...").
            test_filename: Optional .txt path containing test data.

        Returns:
            A tuple of 3-elements or 4-elements, the later iff test_filename is given.
            The first 2 elements are term_index and tag_index, which are dictionaries,
            respectively, from term to integer ID and from tag to integer ID. The int
            IDs are used in the numpy matrices.
            The 3rd element is a tuple itself, consisting of 3 numpy arrsys:
                - train_terms: numpy int matrix.
                - train_tags: numpy int matrix.
                - train_lengths: numpy int vector.
                These 3 are identical to what is returned by BuildMatrices().
            The 4th element is a tuple of 3 elements as above, but the data is
            extracted from test_filename.
        """
        term_index = {'__oov__': 0}  # Out-of-vocab is term 0.
        tag_index = {}
        
        train_data = DatasetReader.ReadFile(train_filename, term_index, tag_index)
        train_terms, train_tags, train_lengths = DatasetReader.BuildMatrices(train_data)
        
        if test_filename:
            test_data = DatasetReader.ReadFile(test_filename, term_index, tag_index)
            test_terms, test_tags, test_lengths = DatasetReader.BuildMatrices(test_data)

            if test_tags.shape[1] < train_tags.shape[1]:
                diff = train_tags.shape[1] - test_tags.shape[1]
                zero_pad = numpy.zeros(shape=(test_tags.shape[0], diff), dtype='int64')
                test_terms = numpy.concatenate([test_terms, zero_pad], axis=1)
                test_tags = numpy.concatenate([test_tags, zero_pad], axis=1)
            elif test_tags.shape[1] > train_tags.shape[1]:
                diff = test_tags.shape[1] - train_tags.shape[1]
                zero_pad = numpy.zeros(shape=(train_tags.shape[0], diff), dtype='int64')
                train_terms = numpy.concatenate([train_terms, zero_pad], axis=1)
                train_tags = numpy.concatenate([train_tags, zero_pad], axis=1)

            return (term_index, tag_index,
                            (train_terms, train_tags, train_lengths),
                            (test_terms, test_tags, test_lengths))
        else:
            return term_index, tag_index, (train_terms, train_tags, train_lengths)


class SequenceModel(object):

    def __init__(self, max_length=310, num_terms=1000, num_tags=40):
        """Constructor. You can add code but do not remove any code.

        The arguments are arbitrary: when you are training on your own, PLEASE set
        them to the correct values (e.g. from main()).

        Args:
            max_lengths: maximum possible sentence length.
            num_terms: the vocabulary size (number of terms).
            num_tags: the size of the output space (number of tags).

        You will be passed these arguments by the grader script.
        """
        self.max_length = max_length
        self.num_terms = num_terms
        self.num_tags = num_tags
        self.x = tf.placeholder(tf.int64, [None, self.max_length], 'X')
        self.lengths = tf.placeholder(tf.int32, [None], 'lengths')
        
        self.is_training = tf.placeholder(tf.bool, [])




    # TODO(student): You must implement this.
    def lengths_vector_to_binary_matrix(self, length_vector):
        """Returns a binary mask (as float32 tensor) from (vector) int64 tensor.
        
        Specifically, the return matrix B will have the following:
            B[i, :lengths[i]] = 1 and B[i, lengths[i]:] = 0 for each i.
        However, since we are using tensorflow rather than numpy in this function,
        you cannot set the range as described.
        
        
        """
        return tf.sequence_mask(length_vector,maxlen=self.max_length,dtype=tf.float32,name=None)

        #return tf.ones([tf.shape(length_vector), self.max_length], dtype=tf.float32)

    # TODO(student): You must implement this.
    def save_model(self, filename):
        """Saves model to a file."""
        pass

    # TODO(student): You must implement this.
    def load_model(self, filename):
        """Loads model from a file."""
        pass

    # TODO(student): You must implement this.
    def build_inference(self):
        

        
        embeddings = tf.get_variable('embeddings1',[self.num_terms, 55],initializer=None,trainable=True)
        x_embeddings = tf.nn.embedding_lookup(embeddings, self.x) 
        print "i want",self.num_terms
        
        
        write_graph("step_1")
  
    
        
        #batch=tf.shape(self.x)[0]
        #lstm_cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True) # can use tf.nn.rnn_cell.GRUCell or tf.nn.rnn_cell.BasicRNNCell instead 
        
        #print lstm_cell.shape
        #cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * 3, state_is_tuple=True)
        
      
        #init_state = cells.zero_state(batch,tf.float32)
       
        #stackedstates, final_state = tf.nn.dynamic_rnn(cells, x_embeddings, initial_state=init_state, sequence_length=self.lengths)
        
      


   

        # create a RNN cell composed sequentially of a number of RNNCells

        # 'outputs' is a tensor of shape [batch_size, max_time, 256]
        # 'state' is a N-tuple where N is the number of LSTMCells containing a
        # tf.contrib.rnn.LSTMStateTuple for each cell
        
  
        state_size=110
        
        

        with tf.name_scope("bidirectional") as scope:
            cell = tf.contrib.rnn.BasicLSTMCell(num_units=state_size)
            (output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell,cell_bw=cell,inputs=x_embeddings,sequence_length=self.lengths,dtype=tf.float32,scope=scope)
            x_embeddings=tf.concat([output_fw, output_bw], axis=-1)



            
        write_graph("step_2")
        stackedstates=x_embeddings
     
        #rnn_cell = tf.keras.layers.SimpleRNNCell(state_size)
        
        #states=[]
        
        
        #cur_state = tf.zeros(shape=[1, state_size])
        #for i in xrange(self.max_length):

            #cur_state = rnn_cell(x_embeddings[:, i, :], [cur_state])[0]  # shape (batch, state_size) 
            #states .append(cur_state)
                    

        #stackedstates = tf.stack(states, axis=1)
      
        
        with tf.name_scope('logits'):
            logits = tf.reshape(stackedstates, [(tf.shape(self.x)[0]*self.max_length),2*state_size])
    
        
  
            logits=tf.layers.dense(inputs=logits,units=self.num_tags)      
        
   
            self.logits = tf.reshape(logits, [tf.shape(self.x)[0],self.max_length, self.num_tags])

        


        write_graph("step_3")

      
        
        
        #self.logits=tf.nn.relu(self.logits)

        #weights_regularizer=None,biases_initializer=None,biases_regularizer=None)
        #logits=tf.contrib.layers.fully_connected(stackedstates,self.num_tags)       
        #predictions = tf.argmax(logits, axis=2, name="predictions")
        """Build the expression from (self.x, self.lengths) to (self.logits).        
           Please do not change or override self.x nor self.lengths in this function.

        Hint:
            - Use lengths_vector_to_binary_matrix
            - You might use tf.reshape, tf.cast, and/or tensor broadcasting.
        """
        # TODO(student): make logits an RNN on x.
        
        
        

        #self.logits = tf.zeros([tf.shape(self.x)[0], self.max_length, self.num_tags])

    # TODO(student): You must implement this.
    def run_inference(self, terms, lengths):
        
        

        
       
        
        logits = self.sess.run(self.logits, {self.x: terms, self.lengths: lengths,self.is_training:False})

        return numpy.argmax(logits,axis=-1)
        """Evaluates self.logits given self.x and self.lengths.
        
        Hint: This function is straight forward and you might find this code useful:
        # logits = session.run(self.logits, {self.x: terms, self.lengths: lengths})
        # return numpy.argmax(logits, axis=2)

        Args:
            terms: numpy int matrix, like terms_matrix made by BuildMatrices.
            lengths: numpy int vector, like lengths made by BuildMatrices.

        Returns:
            numpy int matrix of the predicted tags, with shape identical to the int
            matrix tags i.e. each term must have its associated tag. The caller will
            *not* process the output tags beyond the sentence length i.e. you can have
            arbitrary values beyond length.
        """
        #return numpy.zeros_like(terms)

    # TODO(student): You must implement this.
    def build_training(self):
                            
                    
        
        self.labels=tf.placeholder(tf.int32,[None,self.max_length],name='labels')
                            
        with tf.name_scope('loss'):
        


            self.loss=tf.contrib.seq2seq.sequence_loss(self.logits,self.labels,self.lengths_vector_to_binary_matrix(self.lengths))
        
        

       
            self.learning_rate = tf.placeholder_with_default(numpy.array(0.01, dtype='float32'), shape=[], name='None')      
            
     
            tf.losses.add_loss(self.loss,loss_collection=tf.GraphKeys.LOSSES)
     

        with tf.name_scope('train_step'):
            self.opt = tf.train.AdamOptimizer(self.learning_rate)
            self.train_opt = tf.contrib.training.create_train_op(self.loss,self.opt)
                                                 
                                             
        write_graph("step_4")

        """Prepares the class for training.
        
        It is up to you how you implement this function, as long as train_on_batch
        works.
        
        Hint:
            - Lookup tf.contrib.seq2seq.sequence_loss 
            - tf.losses.get_total_loss() should return a valid tensor (without raising
                an exception). Equivalently, tf.losses.get_losses() should return a
                non-empty list.
                
                
        """

        self.train = tf.summary.scalar("loss", self.loss)

        
       
        
        self.train_summ = tf.summary.scalar("seq_to_seq_Loss", self.loss)
                            
        self.sess = tf.Session()

        self.sess.run(tf.global_variables_initializer())
        
        self.step=1
        self.s=0

        self.train_writer = tf.summary.FileWriter(os.path.join('log', 'train'), self.sess.graph)


        




        pass

    def train_epoch(self, terms, tags, lengths, batch_size=32, learn_rate=1e-2):
        
        
        
        
        indices = numpy.random.permutation(terms.shape[0])
        
        
        learn_rate=learn_rate/self.step*2
       
 
   
            
        self.step=self.step+1
       
        
        
        
        for si in range(0, terms.shape[0], batch_size):
            se = min(si + batch_size, terms.shape[0])
            slice_x = terms[indices[si:se]] # + 0 to copy slice
            slice_y=tags[indices[si:se]]
            l=lengths[indices[si:se]]
            
            
            
            _, summ=self.sess.run([self.train_opt,self.train_summ],{
                self.x: slice_x,
                self.labels: slice_y,
                self.lengths:l,
                self.is_training:True,
                self.learning_rate:learn_rate})
                    
                            

            self.train_writer.add_summary(summ, self.s)
            self.s += 1
           
        
          
        self.train_writer.close()
            
            #print self.sess.run([self.loss],feed_dict={self.x: slice_x,self.labels: slice_y,self.lengths:l,})
        
        """Performs updates on the model given training training data.
        
        This will be called with numpy arrays similar to the ones created in 
        Args:
            terms: int64 numpy array of size (# sentences, max sentence length)
            tags: int64 numpy array of size (# sentences, max sentence length)
            lengths:
            batch_size: int indicating batch size. Grader script will not pass this,
                but it is only here so that you can experiment with a "good batch size"
                from your main block.
            learn_rate: float for learning rate. Grader script will not pass this,
                but it is only here so that you can experiment with a "good learn rate"
                from your main block.

        Return:
            boolean. You should return True iff you want the training to continue. If
            you return False (or do not return anyhting) then training will stop after
            the first iteration!
        """
        
        # <-- Your implementation goes here.
        # Finally, make sure you uncomment the `return True` below.
        return True
        pass

    # TODO(student): You can implement this to help you, but we will not call it.
    def evaluate(self, terms, tags, lengths):
        
        predicted_tags = self.run_inference(terms, lengths)
        
        if predicted_tags is None:
            print('Is your run_inference function implented?')
            return 0
        
        test_accuracy = numpy.sum(numpy.cumsum(numpy.equal(tags, predicted_tags), axis=1)[numpy.arange(lengths.shape[0]),lengths-1])/numpy.sum(lengths + 0.0)
        print test_accuracy
        pass


def main():
    """This will never be called by us, but you are encouraged to implement it for
    local debugging e.g. to get a good model and good hyper-parameters (learning
    rate, batch size, etc)."""
    # Read dataset.
    reader = DatasetReader
    train_filename = 'hmm-training-data/ja_gsd_train_tagged.txt'
    test_filename = 'hmm-training-data/ja_gsd_dev_tagged.txt'
    term_index, tag_index, train_data, test_data = reader.ReadData(train_filename, test_filename)
    (train_terms, train_tags, train_lengths) = train_data
    (test_terms, test_tags, test_lengths) = test_data

    model = SequenceModel(train_tags.shape[1], len(term_index), len(tag_index))
    model.build_inference()
    model.build_training()
    for j in xrange(8):
        start = time.time()
        model.train_epoch(train_terms, train_tags, train_lengths)        
        print('Finished epoch %i. Evaluating ...' % (j+1))
        end = time.time()
        print(end - start)
        model.evaluate(test_terms, test_tags, test_lengths)


    


if __name__ == '__main__':
    main()









....slength [36 15 31 ... 28 34 35]
....slength [21 20 13 10 17 44 67 13 23 35 13 34 11 62 29 13 27 20  9 20 30 15 19 30
 19 18 21  5 29 18  6 14 22 41 32 16 10 74  9 57 26 15 18 21 45 12 17 46
 20 13 16 25 22 23 33  8 16 53 16 33 25  5  8 31 13 29  7 14 11 30 14 12
 33 19 13 39 12 31 27  7 26 20 19 29 14 54 21 21  9 24 33 12 25 47 13 12
  9 19 27 30 49 27 19 29 19 14  6  8 17 49 18 28 90 26 39 13 22 36  9 27
 27  9 12  4 13 19 11 19 20  7  8 52 27 39 11 29  6 19 28  6 14 15 15 34
 25  6 38 12 28 21  6 27 18 23 35 23 37 26  9 10  7 21 12 38 48  7 12 18
 26  5 66 41 27 31 23 51  7 45 17 34 27 10 10 16 21 22  6 12 19 30 26 23
 20 41  5 25 20 12  2 34  9  9  7 17 15 18 15 24 23 12 16 25  8 19 19 33
 22 85 17 27 21  4 39 30 10 40 15 33 18 44  8 31 21 16 10 56 33 12 33 22
 28 74 16 18  6 10 28 16 27 61 47 12 20 35 15  6 34 16 15  9 23 18 12 21
 17 11 25 14 34  9 14  9 39 10 63 15 10  3 30 31 40 14 29 26  6 16 17 46
 35 14 58 37 47 31 19 13 31 29  7 29 14 26 14 20 32 37 13 17 32 22 14  6
 38